In [1]:
from scipy.stats import expon
import numpy as np
from math import factorial
import matplotlib.pyplot as plt
import dill
from astropy import visualization as vis
from gammapy.data import DataStore, Observation
import scanner_interpretation as scani

with open("pkl_jugs/reduced_with_significance.pkl","rb") as f:
    reduced = dill.load(f)

pval_0_mask = reduced["LAMBDA_RATIO_SIGNIFICANCE"]==0.


In [3]:
reduced[pval_0_mask][35]

Nmax,OBS_ID,ID,RA,DEC,TIME,ENERGY,dt,da,MEDIAN_RA,MEDIAN_DEC,SkyCoord,MEDIAN_GLAT,MEDIAN_GLON,TEVCAT_SOURCE_NAME,TEVCAT_SOURCE_TYPE,TEVCAT_DISTANCES_DEG,DS_INDEX,ANGULAR_MEASURE_DEG,BKG_DT_LAMBDA,BKG_PHOTONS,MPLET_DT_LAMBDA,LAMBDA_RATIO_SIGNIFICANCE
,,,,,,,,,,,"deg,deg",deg,deg,,,,,,,,,
int64,uint32,object,object,object,object,object,float64,float64,float64,float64,SkyCoord,float64,float64,str21,str22,float64,int64,float64,float64,int64,float64,float64
3,64292,[30309584208454 30309584208612 30309584208743],[283.17816 283.07266 283.09836],[-30.66926 -30.645079 -30.639166],['2011-05-30T01:43:29.620384747' '2011-05-30T01:43:30.171913916'\n '2011-05-30T01:43:30.658502156'],[0.49678615 0.23055033 1.2978227 ],1038117376.0,0.1082350863608515,283.0983581542969,-30.645078659057617,"283.0983581542969,-30.645078659057617",-13.627917271879404,5.215087184571037,HESS J1800-240A,SNR/Molec. Cloud,14.26931051152663,0,0.03360490257602158,0.0,0,1.926564358386557e-09,0.0


In [2]:
with open("pkl_jugs/plotter_pval0/multiplets.pkl","wb") as f:
    dill.dump(reduced[pval_0_mask],f)

## badly marked: 18361

In [ ]:
reduced[pval_0_mask]["OBS_ID"].data

In [ ]:
h = vis.hist(reduced["ANGULAR_MEASURE_DEG"],histtype="step",bins="freedman")
plt.title("Mean of multiplet photon distance to median coordinate")
plt.xlabel("Distance [deg]")
plt.ylabel("Counts")

In [ ]:
min(reduced["LAMBDA_RATIO_SIGNIFICANCE"][~pval_0_mask]),max(reduced["LAMBDA_RATIO_SIGNIFICANCE"])

Significance histogram excluding pval==0

In [ ]:
h = vis.hist(np.log10(reduced[~pval_0_mask]["LAMBDA_RATIO_SIGNIFICANCE"]),histtype="step",bins=np.linspace(-14,0))
plt.yscale("log")
plt.xlabel(r"$^{10}\log(p)$")
plt.ylabel("Counts")
plt.title("Poissonian Multiplet significance based on bkg and multiplet rates.")
plt.savefig("figures/combined/lambda_ratio_significance.pdf")
plt.savefig("figures/combined/lambda_ratio_significance.png",facecolor="white")
# h = vis.hist(reduced["MPLET_DT_LAMBDA"],histtype="step",bins=np.linspace(6e-19,9.5e-19,100))

In [ ]:
reduced.sort('dt')
pval_0_mask = reduced["LAMBDA_RATIO_SIGNIFICANCE"]==0.

len(np.unique(reduced[pval_0_mask]["OBS_ID"]))

In [ ]:
h = vis.hist(reduced["dt"][reduced["LAMBDA_RATIO_SIGNIFICANCE"] < 1e-5],histtype="step",bins="freedman")
# reduced[(reduced["LAMBDA_RATIO_SIGNIFICANCE"] < 1e-7)*(reduced["dt"] > 27e8)]

In [ ]:
candidates = reduced[reduced["POISSON_SIGNIFICANCE"] < 1e-5]

### To do

1. Check fit compared to histogram and mplet dt
2. Verify on a skymap that the masked photons correspond to near the source
3. Do a time spread of the photon arrival times to guess if a cloud is passing by
4. Do a gammapy analysis
5. Ask victor if the m87 run is excluded (done)
6. check GRBCAT

In [ ]:
ds = scani.getDataStores()
obs = [myds.get_observations(candidates["OBS_ID"],skip_missing=True) for myds in ds]
m87_obs = obs[1][0]
m87_mplet = candidates[-1]


In [ ]:
m87_mplet

In [ ]:
m87_rundist = scani.sphere_dist(m87_obs.events.table["RA"].data,m87_obs.events.table["DEC"].data,m87_mplet["MEDIAN_RA"],m87_mplet["MEDIAN_DEC"])

plt.scatter(1e-9*(m87_obs.events.table[m87_rundist<.1]["TIME"]),np.zeros_like(m87_obs.events.table[m87_rundist<.1]["TIME"]),marker="v",s=1)

In [ ]:
expon.fit(np.diff(np.sort(m87_obs.events.table[m87_rundist < 0.1]["TIME"])),floc=0)

In [ ]:
dwarf = SkyCoord(283.775,-30.483,frame="fk5",unit="deg")
dwarf.separation(reduced[(reduced["OBS_ID"]==64292) * (reduced["POISSON_SIGNIFICANCE"] < 1e-5)]["SkyCoord"])

In [ ]:
h = vis.hist(np.log10(reduced["dt"]),bins="freedman",histtype="step")
plt.yscale("log")

In [ ]:
N = 3
livetime = 1680
bursttime = 3
rate = N/livetime

lamb = rate*bursttime
sum = 1
for i in range(3):
    sum -= (lamb)**i * np.exp(-lamb) / factorial(i)

sum
